# Using Meeshkan for developing Kaggle kernels
For a full example of how to, for example, submit kernels from the command-line, see the example page in [GitHub](https://github.com/Meeshkan/meeshkan-client/tree/kaggle-kernel-example/examples/kaggle).

### Install dependencies

In [1]:
!pip install keras tensorflow meeshkan

In [2]:
import os
import meeshkan
import keras

Using TensorFlow backend.


Check that datasources are correctly linked:

In [3]:
print(os.listdir("../input"))

['test_images', 'breed_labels.csv', 'test_metadata.zip', 'test_images.zip', 'test', 'train_images.zip', 'train_metadata', 'color_labels.csv', 'test_sentiment', '.gitkeep', 'test_metadata', 'test.zip', 'train_sentiment', 'test_sentiment.zip', 'train_sentiment.zip', 'train', 'filename_without_extension', 'train_metadata.zip', 'train_images', 'state_labels.csv', 'train.zip']


### Setting up Meeshkan

To use Meeshkan, you need to 
1. register at [meeshkan.com](https://meeshkan.com) and get our API key
1. setup Slack integration as instructed in the [docs](https://www.meeshkan.com/docs)
1. initialize the credentials and start the agent.

The second step can be achieved with the `meeshkan.init(token=YOUR_API_KEY)` command. **Make sure to replace the example API key below with your own key!**

In [4]:
meeshkan.init(token="meVLHpGVcpG1QI2jTo1VLJpv96ZRTijCROaj4CQSU1IMQ")  # REPLACE THIS WITH YOUR API_KEY

Stopping service...
Service started.


Meeshkan agent can be used to schedule multiple machine learning jobs, but as we're just running stuff sequentially, we can simply use the [as_blocking_job](https://meeshkan-client.readthedocs.io/en/latest/#meeshkan.as_blocking_job) decorator to group runs as jobs. Execute the cell below to ensure you get Slack notifications.

In [5]:
import time
@meeshkan.as_blocking_job(job_name="test-job", report_interval_secs=10)
def train():
    for i in range(10):
        meeshkan.report_scalar("counter", i)
        time.sleep(2)
        
train()

If everything works as expected, we can get rocking with machine learning!

### Run your job

### Teardown

Finally we stop the Meeshkan agent cleanly:

In [6]:
meeshkan.stop()

Stopping service...
Service stopped.
